# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
contents = open("scraped/hm.csv").read()
contents[0:500]

'name,price,url\nWashed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N\nCandle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D\nGlittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A\nTextured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C\nStoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A\nSlub-weave Cushion Cover,$3.99 $9.99,http://www.h'

In [3]:
df = pd.read_csv('scraped/hm.csv')
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [4]:
df['price']

0       $59.99 $129
1      $6.99 $17.99
2      $7.99 $17.99
3      $6.99 $12.99
4     $17.99 $24.99
5       $3.99 $9.99
6      $7.99 $17.99
7      $7.99 $17.99
8      $9.99 $17.99
9     $27.99 $34.99
10      $2.99 $5.99
11      $2.99 $5.99
12     $9.99 $12.99
13    $39.99 $49.99
14    $39.99 $49.99
15     $7.99 $17.99
16    $19.99 $29.99
17      $2.99 $5.99
18     $7.99 $17.99
19    $14.99 $24.99
20     $9.99 $24.99
21     $7.99 $12.99
22     $7.99 $17.99
23      $4.99 $9.99
24     $9.99 $12.99
25     $7.99 $17.99
26      $3.99 $6.99
27     $7.99 $17.99
28      $2.99 $6.99
29     $9.99 $12.99
30    $14.99 $34.99
31     $7.99 $17.99
32    $24.99 $49.99
33    $29.99 $79.99
34     $4.99 $12.99
35    $19.99 $24.99
36      $2.99 $5.99
37    $14.99 $34.99
38     $6.99 $17.99
39     $6.99 $17.99
40     $6.99 $17.99
41         $99 $199
42       $54.99 $99
43     $6.99 $17.99
44     $4.99 $12.99
45     $7.99 $17.99
46      $3.99 $9.99
47     $6.99 $17.99
48     $4.99 $12.99
49      $2.99 $5.99


In [5]:
df['price_original'] = df['price'].str.extract(r"\s\$(.*)")

In [6]:
df['price_original']

0       129
1     17.99
2     17.99
3     12.99
4     24.99
5      9.99
6     17.99
7     17.99
8     17.99
9     34.99
10     5.99
11     5.99
12    12.99
13    49.99
14    49.99
15    17.99
16    29.99
17     5.99
18    17.99
19    24.99
20    24.99
21    12.99
22    17.99
23     9.99
24    12.99
25    17.99
26     6.99
27    17.99
28     6.99
29    12.99
30    34.99
31    17.99
32    49.99
33    79.99
34    12.99
35    24.99
36     5.99
37    34.99
38    17.99
39    17.99
40    17.99
41      199
42       99
43    17.99
44    12.99
45    17.99
46     9.99
47    17.99
48    12.99
49     5.99
50     9.99
51    12.99
52     5.99
53    79.99
54    17.99
55    79.99
56     5.99
57    24.99
58    12.99
59    17.99
Name: price_original, dtype: object

In [7]:
df['price_discounted'] = df['price'].str.extract(r"\$(.* )")

In [8]:
df['price_discounted']

0     59.99 
1      6.99 
2      7.99 
3      6.99 
4     17.99 
5      3.99 
6      7.99 
7      7.99 
8      9.99 
9     27.99 
10     2.99 
11     2.99 
12     9.99 
13    39.99 
14    39.99 
15     7.99 
16    19.99 
17     2.99 
18     7.99 
19    14.99 
20     9.99 
21     7.99 
22     7.99 
23     4.99 
24     9.99 
25     7.99 
26     3.99 
27     7.99 
28     2.99 
29     9.99 
30    14.99 
31     7.99 
32    24.99 
33    29.99 
34     4.99 
35    19.99 
36     2.99 
37    14.99 
38     6.99 
39     6.99 
40     6.99 
41       99 
42    54.99 
43     6.99 
44     4.99 
45     7.99 
46     3.99 
47     6.99 
48     4.99 
49     2.99 
50     4.99 
51     6.99 
52     2.99 
53    34.99 
54     7.99 
55    34.99 
56     2.99 
57    14.99 
58     5.99 
59    12.99 
Name: price_discounted, dtype: object

In [9]:
df.dtypes

name                object
price               object
url                 object
price_original      object
price_discounted    object
dtype: object

In [10]:
df['price_original'] = df.price_original.astype(float)
df['price_discounted'] = df.price_discounted.astype(float)
df.dtypes

name                 object
price                object
url                  object
price_original      float64
price_discounted    float64
dtype: object

In [11]:
df.head()

,name,price,url,price_original,price_discounted
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99


In [12]:
df.pct_discount = round((df.price_original - df.price_discounted)/df.price_original*100,1)
df['pct_discount'] = df.pct_discount

C:\Users\kellykiki\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [13]:
df

,name,price,url,price_original,price_discounted,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99,53.5
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99,61.1
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99,55.6
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99,46.2
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99,28.0
5,Slub-weave Cushion Cover,$3.99 $9.99,http://www.hm.com/us/product/70965?article=70965-D,9.99,3.99,60.1
6,Braided Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/62818?article=62818-B,17.99,7.99,55.6
7,Jacquard-weave Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/69163?article=69163-B,17.99,7.99,55.6
8,Scented Candle in Glass Holder,$9.99 $17.99,http://www.hm.com/us/product/40910?article=40910-C,17.99,9.99,44.5
9,2-pack Curtain Panels,$27.99 $34.99,http://www.hm.com/us/product/69699?article=69699-B,34.99,27.99,20.0


In [14]:
df['article_id'] = df['url'].str.extract(r"\=(\d+-\w)")
df['article_id']

0     13472-N
1     35079-D
2     72462-A
3     58926-C
4     74242-A
5     70965-D
6     62818-B
7     69163-B
8     40910-C
9     69699-B
10    76916-A
11    68473-C
12    74498-A
13    41512-E
14    76727-A
15    76977-A
16    73369-A
17    59417-D
18    76797-A
19    76794-A
20    80432-A
21    78352-A
22    70658-A
23    72111-A
24    77665-A
25    70603-A
26    72262-B
27    70427-A
28    72262-A
29    77665-B
30    73380-A
31    35079-C
32    74773-A
33    75033-A
34    60637-C
35    70306-A
36    60621-B
37    69643-A
38    72455-B
39    72460-B
40    72462-B
41    71871-A
42    73734-A
43    74267-A
44    73344-A
45    73325-C
46    74264-A
47    72455-A
48    70799-B
49    68473-D
50    72111-C
51    58926-B
52    58211-A
53    75469-A
54    76982-A
55    74825-A
56    74263-A
57    78047-A
58    60637-G
59    78381-A
Name: article_id, dtype: object

In [15]:
df

,name,price,url,price_original,price_discounted,pct_discount,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99,53.5,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99,61.1,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99,55.6,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99,46.2,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99,28.0,74242-A
5,Slub-weave Cushion Cover,$3.99 $9.99,http://www.hm.com/us/product/70965?article=70965-D,9.99,3.99,60.1,70965-D
6,Braided Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/62818?article=62818-B,17.99,7.99,55.6,62818-B
7,Jacquard-weave Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/69163?article=69163-B,17.99,7.99,55.6,69163-B
8,Scented Candle in Glass Holder,$9.99 $17.99,http://www.hm.com/us/product/40910?article=40910-C,17.99,9.99,44.5,40910-C
9,2-pack Curtain Panels,$27.99 $34.99,http://www.hm.com/us/product/69699?article=69699-B,34.99,27.99,20.0,69699-B


In [16]:
df.to_csv("hm_cleaned.csv", index=False)


In [17]:
pd.read_csv("hm_cleaned.csv")
df.head()

,name,price,url,price_original,price_discounted,pct_discount,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99,53.5,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99,61.1,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99,55.6,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99,46.2,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99,28.0,74242-A


## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [18]:
contents = open("scraped/sci-fi.csv").read()
contents[0:500]

'full_rating,full_score,rank,title,url\n" 4.07 avg rating β€” 785,502 ratings","\nscore: 28,539,\n              and\n292 people voted\n              β€ƒ\n              \n            ",1,"\nThe Handmaid\'s Tale\n",/book/show/38447.The_Handmaid_s_Tale\n" 4.34 avg rating β€” 5,212,935 ratings","\nscore: 27,566,\n              and\n282 people voted\n              β€ƒ\n              \n            ",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games\n" 3.76 avg rating β€” 922,308 ratings",'

In [19]:
df = pd.read_csv('scraped/sci-fi.csv')
df.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [20]:
df['title'] = df.title.str.strip()
#df['title']

In [21]:
df

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 people voted\n \n \n",6,"Divergent (Divergent, #1)",/book/show/13335037-divergent
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 people voted\n \n \n",7,"Catching Fire (The Hunger Games, #2)",/book/show/6148028-catching-fire
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 people voted\n \n \n",8,"The Giver (The Giver, #1)",/book/show/3636.The_Giver
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 people voted\n \n \n",9,The Dispossessed,/book/show/13651.The_Dispossessed
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people voted\n \n \n",10,Kindred,/book/show/60931.Kindred


In [22]:
#df.full_rating

In [23]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows',100)

In [24]:
df['avg_rating'] = df['full_rating'].str.extract(r"(\d*.\d*) avg")
#df['avg_rating']

In [25]:
df['rating_count'] = df['full_rating'].str.extract(r"(\d*,*\d*) ratings")
#df['rating_count']

In [26]:
df['total_score'] = df['full_score'].str.extract(r"(\d+,\d+),*")
#df['total_score']

In [27]:
df['score_votes'] = df['full_score'].str.extract(r"(\d*) people")
#df['score_votes']

In [28]:
#df.full_score

In [29]:
#df.title

In [30]:
df['series'] = df['title'].str.extract(r"[(](.*),")
#df['series']

In [31]:
df['series_no'] = df['title'].str.extract(r"#(\d.*\d*)[)]")
df.series_no.replace("#"," ")

0     NaN     
1     1       
2     NaN     
3     1       
4     NaN     
5     1       
6     2       
7     1       
8     NaN     
9     NaN     
10    1       
11    3       
12    NaN     
13    1       
14    NaN     
15    1       
16    1       
17    1       
18    1       
19    1       
20    2       
21    2       
22    7       
23    1       
24    2       
25    NaN     
26    10      
27    1       
28    1       
29    2       
30    8       
31    1       
32    6       
33    1-3     
34    5       
35    2       
36    12      
37    9       
38    1       
39    1       
40    1       
41    1       
42    4       
43    1       
44    3       
45    1       
46    NaN     
47    3       
48    1       
49    11      
50    1       
51    1       
52    1-3     
53    NaN     
54    NaN     
55    1       
56    5.1-5.3 
57    1       
58    1       
59    1       
60    1       
61    NaN     
62    1       
63    1       
64    2       
65    13      
66    6609

In [32]:
df

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"212,935","27,566",282,The Hunger Games,1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185,A Wrinkle in Time Quintet,1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165,NaN,NaN
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 people voted\n \n \n",6,"Divergent (Divergent, #1)",/book/show/13335037-divergent,4.23,"345,974","12,935",134,Divergent,1
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 people voted\n \n \n",7,"Catching Fire (The Hunger Games, #2)",/book/show/6148028-catching-fire,4.30,"049,239","12,261",128,The Hunger Games,2
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 people voted\n \n \n",8,"The Giver (The Giver, #1)",/book/show/3636.The_Giver,4.12,"379,452","11,238",117,The Giver,1
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 people voted\n \n \n",9,The Dispossessed,/book/show/13651.The_Dispossessed,4.19,"57,605","10,246",107,NaN,NaN
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people voted\n \n \n",10,Kindred,/book/show/60931.Kindred,4.20,"53,473","9,907",104,NaN,NaN


In [33]:
df.to_csv("sci-fi_cleaned.csv", index=False)

In [34]:
pd.read_csv("sci-fi_cleaned.csv")
df.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"212,935","27,566",282,The Hunger Games,1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185,A Wrinkle in Time Quintet,1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165,NaN,NaN


## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [35]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [36]:
df.content

0                                                                   \n\n                           DEPARTMENT OF DEFENSE \n         DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n           \n             \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\n \n\nCOACHER, Robert E., Administrative Judge: \n\n \nApplicant has not mitigated the alcohol consumption security concerns. Eligibility \n\nfor access to classified information is denied.  \n\nStatement of the Case \n\nOn  June  16,  2015,  the  Department  of  Defense  Consolidated  Adjudications \nFacility (DOD CAF) issued Applicant a Statement of Reasons (SOR) detailing security \nconcerns  under  Guideline  G,  alcohol  consumption.  DOD  CAF  acted  under  Executive \nOrder  (EO)  10865

Okay, now do the work and **make those new columns!**

In [37]:
df['lawyer_app'] = df.content.str.extract(r"For Applicant[:] *(.*).*", re.IGNORECASE)
#df

In [38]:
df.lawyer_app

0     Pro se                  
1     Mark S. Zaid, Esq.      
2     Pro se                  
3     Mark S. Zaid, Esq.      
4     Pro se                  
5     Pro se                  
6     NaN                     
7     Ryan C. Nerney, Esquire 
8     Pro se                  
9     Pro se                  
10    Pro Se                  
11    NaN                     
12    Pro se                  
13    Stephen Glassman, Esq.  
14    Pro se                  
Name: lawyer_app, dtype: object

In [39]:
#df.content[6]
#df.content[11]
#n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n
#\n\nFOR APPLICANT\n\nPro se\n\n

In [40]:
df['lawyer_gov'] = df.content.str.extract(r"For Government[:]*[\n ]*(.*).*, Esq.*", re.IGNORECASE)

In [41]:
df.lawyer_gov

0     Stephanie C. Hess  
1     Robert J. Kilmartin
2     Richard Stevens    
3     Julie R. Mendez    
4     Julie R. Mendez    
5     David F. Hayes     
6     James B. Norman    
7     Richard Stevens    
8     Tara Karoian       
9     Eric Borgstrom     
10    Robert Kilmartin   
11    James B. Norman    
12    Tovah Minster      
13    Erin Thompson      
14    David F. Hayes     
Name: lawyer_gov, dtype: object

In [42]:
df['judge'] = df.content.str.extract(r"(.*.*), Administrative Judge", re.IGNORECASE)

In [43]:
df.judge

0     COACHER, Robert E.      
1     LOUGHRAN, Edward W.     
2     DUFFY, James F.         
3     HARVEY, Mark            
4     MARSHALL, Jr., Arthur E.
5     MENDEZ, Francisco       
6     NaN                     
7     GALES, Robert Robinson  
8     DUFFY, James F.         
9     HOGAN, Erin C.          
10    GOLDSTEIN, Jennifer I.  
11    NaN                     
12    MOGUL, Martin H.        
13    GARCIA, Candace Le’i    
14    HOWE, Philip S.         
Name: judge, dtype: object

In [44]:
#df['judge_test'] = df.content.str.extract(r"\n(.*)Administrative Judge", re.IGNORECASE) 

In [45]:
#df.judge_test

In [46]:
df

filename  \
0   pdfs\11-02438.h1.pdf.txt   
1   pdfs\11-03073.h1.pdf.txt   
2   pdfs\11-04909.h1.pdf.txt   
3   pdfs\11-07728.h1.pdf.txt   
4   pdfs\11-08313.h1.pdf.txt   
5   pdfs\11-11916.h1.pdf.txt   
6   pdfs\11-12537.a1.pdf.txt   
7   pdfs\11-12635.h1.pdf.txt   
8   pdfs\11-14135.h1.pdf.txt   
9   pdfs\11-14832.h1.pdf.txt   
10  pdfs\12-00038.h1.pdf.txt   
11  pdfs\12-01601.a1.pdf.txt   
12  pdfs\12-01601.h1.pdf.txt   
13  pdfs\12-01787.h1.pdf.txt   
14  pdfs\12-01969.h1.pdf.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [47]:
df.content.str.extract(r" is (denied|granted)", re.IGNORECASE) # Please see the next cell 

,0
0,denied
1,granted
2,granted
3,granted
4,denied
5,denied
6,NaN
7,granted
8,denied
9,granted


In [48]:
#With a different search, we get clearance desicions which were previously missing. 
#We confirm results for the rest of the cases.
df['access'] = df.content.str.extract(r"(denied|granted)", re.IGNORECASE)
df.access

0     denied 
1     granted
2     granted
3     granted
4     denied 
5     denied 
6     denied 
7     granted
8     denied 
9     granted
10    denied 
11    denied 
12    denied 
13    granted
14    denied 
Name: access, dtype: object

In [49]:
df

filename  \
0   pdfs\11-02438.h1.pdf.txt   
1   pdfs\11-03073.h1.pdf.txt   
2   pdfs\11-04909.h1.pdf.txt   
3   pdfs\11-07728.h1.pdf.txt   
4   pdfs\11-08313.h1.pdf.txt   
5   pdfs\11-11916.h1.pdf.txt   
6   pdfs\11-12537.a1.pdf.txt   
7   pdfs\11-12635.h1.pdf.txt   
8   pdfs\11-14135.h1.pdf.txt   
9   pdfs\11-14832.h1.pdf.txt   
10  pdfs\12-00038.h1.pdf.txt   
11  pdfs\12-01601.a1.pdf.txt   
12  pdfs\12-01601.h1.pdf.txt   
13  pdfs\12-01787.h1.pdf.txt   
14  pdfs\12-01969.h1.pdf.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [50]:
df.to_csv("court_cleaned.csv", index=False)

In [51]:
pd.read_csv("court_cleaned.csv")
df.head()

filename  \
0  pdfs\11-02438.h1.pdf.txt   
1  pdfs\11-03073.h1.pdf.txt   
2  pdfs\11-04909.h1.pdf.txt   
3  pdfs\11-07728.h1.pdf.txt   
4  pdfs\11-08313.h1.pdf.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [52]:
austen = open("data/Austen_Pride.txt").read()

## Look at the first 500 or so characters of it 

In [53]:
austen[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [54]:
len(re.findall(r"[\n ](he)\b", austen, re.IGNORECASE))

1308

In [55]:
len(re.findall(r"[\n ](she)\b", austen, re.IGNORECASE))

1682

In [56]:
len(re.findall(r"[\n ](he)\b", austen, re.IGNORECASE))+len(re.findall(r"[\n ](she)\b", austen, re.IGNORECASE))

2990

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [57]:
re.findall(r"[\n ]+he \w*|[\n ]+she \w*", austen, re.IGNORECASE)

[' he is',
 ' he had',
 ' she told',
 ' he came',
 ' he agreed',
 ' he is',
 ' he married',
 ' he may',
 ' he comes',
 ' she ought',
 ' he comes',
 ' he chooses',
 ' she is',
 ' She was',
 ' she was',
 ' she fancied',
 ' He had',
 ' he should',
 ' she had',
 ' he suddenly',
 ' She has',
 ' She is',
 ' she will',
 ' she will',
 ' he continued',
 ' he wished',
 ' she began',
 ' she had',
 ' he spoke',
 ' he left',
 ' he would',
 ' he eluded',
 ' He was',
 ' he meant',
 ' He had',
 ' he had',
 ' he saw',
 ' he wore',
 ' She could',
 ' he could',
 ' she began',
 ' he might',
 ' he ought',
 ' he brought',
 ' he had',
 ' he was',
 ' he was',
 ' he was',
 ' he was',
 ' He was',
 ' he would',
 ' She is',
 ' he looked',
 ' he withdrew',
 ' She told',
 ' she had',
 ' she had',
 ' he was',
 ' he had',
 ' He had',
 ' he ',
 ' he had',
 ' she entered',
 ' she looked',
 ' he actually',
 ' she was',
 ' he asked',
 ' he asked',
 ' he did',
 ' he seemed',
 ' she was',
 ' he inquired',
 ' she was',
 ' h

In [58]:
#len(re.findall(r"[\n ]+he \w*|[\n ]+she \w*", austen, re.IGNORECASE))

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [59]:
he_she = re.findall(r"[\n ]+(he|she) (\w*)", austen, re.IGNORECASE)
he_she

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed'),
 ('he', 'is'),
 ('he', 'married'),
 ('he', 'may'),
 ('he', 'comes'),
 ('she', 'ought'),
 ('he', 'comes'),
 ('he', 'chooses'),
 ('she', 'is'),
 ('She', 'was'),
 ('she', 'was'),
 ('she', 'fancied'),
 ('He', 'had'),
 ('he', 'should'),
 ('she', 'had'),
 ('he', 'suddenly'),
 ('She', 'has'),
 ('She', 'is'),
 ('she', 'will'),
 ('she', 'will'),
 ('he', 'continued'),
 ('he', 'wished'),
 ('she', 'began'),
 ('she', 'had'),
 ('he', 'spoke'),
 ('he', 'left'),
 ('he', 'would'),
 ('he', 'eluded'),
 ('He', 'was'),
 ('he', 'meant'),
 ('He', 'had'),
 ('he', 'had'),
 ('he', 'saw'),
 ('he', 'wore'),
 ('She', 'could'),
 ('he', 'could'),
 ('she', 'began'),
 ('he', 'might'),
 ('he', 'ought'),
 ('he', 'brought'),
 ('he', 'had'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('He', 'was'),
 ('he', 'would'),
 ('She', 'is'),
 ('he', 'looked'),
 ('he', 'withdrew'),
 ('She', 'told'),
 ('she', 'had'),
 ('she', 'h

In [60]:
#len(re.findall(r"[\n ]+(he|she) (\w*)", austen, re.IGNORECASE))

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [61]:
df = pd.DataFrame(he_she, columns=['pronoun', 'verb'])
df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [62]:
df.pronoun.value_counts()

she    1316
he     1048
She    306 
He     212 
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [63]:
df['pronoun']= df.pronoun.str.lower()

In [64]:
df.pronoun.value_counts()

she    1622
he     1260
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [65]:
df.verb.value_counts().sort_values(ascending=False).head(20)

was          371
had          369
could        170
is           116
would        92 
has          65 
did          64 
will         49 
might        46 
should       41 
felt         37 
must         36 
said         33 
thought      31 
added        31 
saw          31 
then         26 
replied      22 
continued    21 
looked       20 
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [66]:
he_verbs = df[df['pronoun']=="he"]
he_verbs.verb.value_counts().sort_values(ascending=False).head(20)

had          165
was          159
is           62 
has          47 
could        39 
would        34 
did          28 
should       26 
will         24 
must         23 
might        21 
replied      14 
thought      13 
said         13 
came         13 
may          12 
never        12 
does         11 
then         11 
continued    10 
Name: verb, dtype: int64

In [67]:
she_verbs = df[df['pronoun']=="she"]
she_verbs.verb.value_counts().sort_values(ascending=False).head(20)

was        212
had        204
could      131
would      58 
is         54 
did        36 
felt       32 
saw        28 
might      25 
will       25 
added      23 
said       20 
has        18 
thought    18 
should     15 
then       15 
must       13 
found      13 
soon       12 
cried      11 
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [68]:
he_cried = he_verbs.verb.value_counts("normalize").loc['cried']
he_cried

0.0007936507936507937

In [69]:
she_cried = she_verbs.verb.value_counts("normalize").loc['cried']
she_cried

0.006781750924784217

In [70]:
print("Austin's women cried more, as they cried by", round(she_cried*100,2), 'but men cried by', round(he_cried*100,2))

Austin's women cried more, as they cried by 0.68 but men cried by 0.08


## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [71]:
tolkein = open("data/Lord_of_the_Rings_01 _The_Fellowship_of_the_Ring_ JRR_Tolkien _1955.txt", encoding="utf-8").read()

In [72]:
tolkein[:500]

'THE FELLOWSHIP OF THE RING\n\n\n\n\nBEING THE FIRST PART OF\n\nTHE LORD OF THE RINGS\n\nBY\n\nJ.R.R. TOLKIEN\n\n\n\n\n\nThree Rings for the Elven-kings under the sky,\n\nSeven for the Dwarf-lords in their halls of stone,\n\nNine for Mortal Men doomed to die,\n\nOne for the Dark Lord on his dark throne\n\nIn the Land of Mordor where the Shadows lie.\n\nOne Ring to rule them all, One Ring to find them,\n\nOne Ring to bring them all and in the darkness bind them\n\nIn the Land of Mordor where the Shadows lie.\n\n\n\n\n\nCONTENTS\n\n\nCOV'

In [73]:
male_female = pd.DataFrame(re.findall(r"[\n ]+he\b|[\n ]+she\b", tolkein, re.IGNORECASE), columns=["pronoun"])
#male_female

In [74]:
male_female['pronoun']= male_female.pronoun.str.lower()

In [75]:
male_female

,pronoun
0,he
1,he
2,he
3,he
4,he
5,he
6,he
7,he
8,he
9,he


In [76]:
male_female['pronoun']= male_female.pronoun.str.strip()


In [77]:
male_female

,pronoun
0,he
1,he
2,he
3,he
4,he
5,he
6,he
7,he
8,he
9,he


In [78]:
male_female.pronoun.value_counts("normalize = True")*100

he     95.047619
she    4.952381 
Name: pronoun, dtype: float64

In [80]:
df.pronoun.value_counts("normalize = True")*100

she    56.280361
he     43.719639
Name: pronoun, dtype: float64